In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- Synthetic dataset ----------------
np.random.seed()
N = 10000

route_planning = pd.DataFrame({
    'origin_x': np.random.uniform(0, 100, N),
    'origin_y': np.random.uniform(0, 100, N),
    'dest_x': np.random.uniform(0, 100, N),
    'dest_y': np.random.uniform(0, 100, N),
    'traffic_density': np.random.uniform(0, 1, N),
    'road_type': np.random.choice([1, 2, 3], N),
})
route_planning['distance'] = np.sqrt((route_planning['dest_x'] - route_planning['origin_x'])**2 +
                                     (route_planning['dest_y'] - route_planning['origin_y'])**2)
speed_base = {1:50, 2:40, 3:30}
route_planning['speed'] = route_planning['road_type'].map(speed_base) * np.random.uniform(0.8,1.2,N)
route_planning['travel_time'] = (route_planning['distance']/route_planning['speed'])*60*\
                                (1+route_planning['traffic_density']*np.random.uniform(0.1,0.5,N))

vehicle_assignment = pd.DataFrame({
    'vehicle_capacity': np.random.randint(50,200,N),
    'battery_level': np.random.uniform(0.3,1.0,N),

    'delivery_size': np.random.randint(5,50,N),
    'vehicle_type': np.random.choice([1,2],N),
    'speed_factor': np.random.uniform(0.9,1.1,N),
})
vehicle_assignment['assigned_speed'] = route_planning['speed']*vehicle_assignment['speed_factor']
vehicle_assignment['load_utilization'] = vehicle_assignment['delivery_size']/vehicle_assignment['vehicle_capacity']

time_scheduling = pd.DataFrame({
    'requested_time': np.random.randint(8,20,N),
    'delivery_priority': np.random.randint(1,5,N),
    'customer_patience': np.random.uniform(0,1,N),
})
time_scheduling['delay_probability'] = np.clip(
    (route_planning['travel_time']/60)*(1+vehicle_assignment['load_utilization']*0.5)*np.random.uniform(0.8,1.2,N),
    0,1
)

dynamic_rerouting = pd.DataFrame({
    'current_x': np.random.uniform(0,100,N),
    'current_y': np.random.uniform(0,100,N),
    'traffic_updates': np.random.uniform(0,1,N),
    'new_delivery_requests': np.random.randint(0,3,N),
    'vehicle_status': np.random.choice([0,1],N),
    'weather': np.random.choice([0,1],N),
})
dynamic_rerouting['congestion_score'] = dynamic_rerouting['traffic_updates'] + \
                                       dynamic_rerouting['new_delivery_requests']*0.5 + \
                                       dynamic_rerouting['weather']*0.5 + \
                                       (route_planning['travel_time']/route_planning['travel_time'].max())*0.5

# ---------------- Combine and normalize ----------------
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting]
dataset_dims = [df.shape[1] for df in datasets]
max_dim = max(dataset_dims)

padded_data = []
for df in datasets:
    arr = df.values
    if arr.shape[1] < max_dim:
        arr = np.hstack([arr, np.zeros((arr.shape[0], max_dim - arr.shape[1]))])
    padded_data.append(arr)

DATA_MATRIX = np.hstack(padded_data)
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0)+1e-8)
def generate_targets(DATA_MATRIX, datasets, candidate_dims, D_graph):
    """
    Generate targets using meaningful metrics for each node.
    Each node aggregates features from multiple datasets.
    """
    targets = []
    N = DATA_MATRIX.shape[0]

    for node_idx in range(D_graph):
        # pick a random row for each dataset
        route_idx = node_idx % len(datasets[0])
        vehicle_idx = node_idx % len(datasets[1])
        time_idx = node_idx % len(datasets[2])
        dynamic_idx = node_idx % len(datasets[3])

        # Construct node vector (can use weighted metrics)
        node_vector = np.hstack([
            datasets[0].iloc[route_idx][['distance','travel_time','speed','traffic_density']].values,
            datasets[1].iloc[vehicle_idx][['load_utilization','assigned_speed','vehicle_capacity']].values,
            datasets[2].iloc[time_idx][['delay_probability','delivery_priority']].values,
            datasets[3].iloc[dynamic_idx][['congestion_score','vehicle_status','weather']].values
        ])

        # Normalize node vector
        node_vector = (node_vector - node_vector.min()) / (np.ptp(node_vector)+1e-8)

        # Sample candidate dimensions
        node_targets = {}
        for dim in candidate_dims:
            if len(node_vector) >= dim:
                sampled = node_vector[:dim]
            else:
                sampled = np.pad(node_vector, (0, dim - len(node_vector)), constant_values=0.5)
            node_targets[dim] = sampled
        targets.append(node_targets)

    return targets

candidate_dims = [len(synthetic_targets[i][list(synthetic_targets[i].keys())[0]])
                  for i in range(D_graph)]
#[6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,]
D_graph = 4
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting]

synthetic_targets = generate_targets(DATA_MATRIX, datasets, candidate_dims, D_graph)


# ---------------- Targets ----------------
#candidate_dims = [6,6,6,6,6,6,6]#,6,6,6,6,6,6,6,6,6,6,6,6,]
#D_graph = 4
inner_archive_size = 120
inner_offspring = 80
outer_archive_size = 80
outer_offspring = 80
inner_iters_per_outer = 15
outer_generations = 500
outer_cost_limit = 1350
inner_learning = 0.9


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- CONFIG ----------------
#candidate_dims = [3] * 50
D_graph = 4

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 1500
outer_cost_limit = 10000
inner_learning = 0.0
gamma_interlayer = 0
top_k = 2630
np.random.seed()


import numpy as np

class MetricsEvaluator:
    def __init__(self, data_matrix, D_graph, metric_keys=['wait','throughput','util','patience']):
        """
        data_matrix: normalized data
        D_graph: number of nodes
        metric_keys: metrics per node
        """
        self.data_matrix = data_matrix
        self.D_graph = D_graph
        self.metric_keys = metric_keys
        self.num_features = data_matrix.shape[1]

        # Initialize learnable parameters per node and per metric
        # Each node has a weight vector (num_features) and bias per metric
        self.params = {}
        for node_idx in range(D_graph):
            self.params[node_idx] = {}
            for key in metric_keys:
                self.params[node_idx][key] = {
                    'weights': np.random.uniform(-1,1,self.num_features),
                    'bias': np.random.uniform(0,1)
                }

    def compute_node_metrics(self, node_idx, y=None):
        """
        Compute metrics using linear combination of features + bias per metric.
        y: optional auxiliary vector to modulate metrics
        """
        row = self.data_matrix[node_idx % self.data_matrix.shape[0]]
        if y is None:
            y = np.zeros(3)
        else:
            y = np.array(y[:3]) if len(y)>=3 else np.pad(y,(0,3-len(y)),constant_values=0.5)

        metrics = {}
        for key in self.metric_keys:
            w = self.params[node_idx][key]['weights']
            b = self.params[node_idx][key]['bias']
            # Linear combination of features, plus optional small modulation from y
            metrics[key] = float(np.dot(row, w) + b + 0.1*np.sum(y))

        # Combine into score
        metrics['score'] = -metrics['wait'] + metrics['throughput'] + metrics['util'] + metrics['patience']
        return metrics

    def compute_all_metrics(self):
        all_metrics = []
        for node_idx in range(self.D_graph):
            all_metrics.append(self.compute_node_metrics(node_idx))
        return all_metrics


# ---------------- INTER-LAYER MUTUAL INFORMATION ----------------
class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.max_input = 2*max_inner_dim
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim if inter_dim is not None else max_inner_dim
        self.weights = {(i,j): np.random.uniform(-0.6,0.6,(self.inter_dim,self.max_input))
                        for i in range(D_graph) for j in range(D_graph) if i!=j}
        self.bias = {(i,j): np.random.uniform(-0.3,0.3,self.inter_dim)
                     for i in range(D_graph) for j in range(D_graph) if i!=j}

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0,self.max_input-len(concat))))[:self.max_input]
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]
        return 1/(1+np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i==j: continue
                if abs(Gmat[i,j])>self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i,j,nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts)<2: return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1,keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2,axis=1)/(A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds,stds)+1e-12)
        np.fill_diagonal(corr,0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat,nested_reps)
        if not acts: return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class FMT:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag

        self.nested_reps = [np.zeros(max(candidate_dims)) for _ in range(D_graph)]
        self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=100, lr_x=0.00001, lr_y=0.00001, lr_W=0.00001):

        # two activation channels
        x = target.copy()
        y = target.copy()

        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target))

        for _ in range(steps):

            z = W.dot(x)

            Theta_grad_z = 2*z - target

            # u = x, v = y
            Theta_grad_x = Theta_grad_z @ W + (y + 1)
            Theta_grad_y = (x + 1)

            Theta_grad_W = np.outer(Theta_grad_z, x)

            # updates
            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)

            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        self.nested_reps[node_idx] = x

        self.l2_after.append(np.linalg.norm(x - target))
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score

    # ---------- OUTER LOOP ----------
    def run_outer(self, outer_cost_limit=1000):
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX, self.D_graph)
        node_metrics_list = []
        raw_scores = []

        # --- Compute node metrics per node ---
        for i, y in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=y)
            node_metrics_list.append(metrics)
            raw_scores.append(metrics['score'])

        raw_scores = np.array(raw_scores)
        total_raw = raw_scores.sum()

        # --- Apply cap to raw metrics ---
        capped_total_raw = total_raw
        if total_raw > outer_cost_limit:
            scale_factor = outer_cost_limit / total_raw
            for metrics in node_metrics_list:
                for key in ['wait', 'throughput', 'util', 'patience', 'score']:
                    metrics[key] *= scale_factor
            raw_scores *= scale_factor
            capped_total_raw = outer_cost_limit

        # --- Compute Fuzzy Metric Tensor contribution ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        D = self.D_graph

        # Only consider off-diagonal entries for inter-node interactions
        off_diag_mask = np.ones((D, D), dtype=bool)
        np.fill_diagonal(off_diag_mask, 0)
        fuzzy_score_offdiag = fuzzy_tensor[off_diag_mask].sum()

        # --- Compute per-node contribution ---
        node_contributions = np.zeros(D)
        for i in range(D):
            # Contribution from own metrics
            own_score = raw_scores[i]

            # Contribution from FMT interactions (row i -> others)
            fmt_contrib = fuzzy_tensor[i, :, :].sum() - fuzzy_tensor[i, i, :].sum()  # exclude self
            node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib

        # --- Combine total score ---
        combined_score = node_contributions.sum()

        # --- Store for plotting / further analysis ---
        self.capped_node_metrics = node_metrics_list
        self.node_score_contributions = node_contributions

        return node_metrics_list, combined_score, node_contributions



    # ---------- FULL RUN ----------
    def run(self, outer_generations=outer_generations):
      best_score = -np.inf
      best_state = {}

      for gen in range(outer_generations):
          mi_scores = []

          # --- Inner loop per node ---
          for node_idx in range(self.D_graph):
              dim = self.best_dim_per_node[node_idx]
              target = self.synthetic_targets[node_idx][dim]
              _, _, _, mi_score = self.run_inner(node_idx, target, dim)
              mi_scores.append(mi_score)

          # --- Outer loop ---
          metrics_list, capped_score, node_contributions = self.run_outer()

          # --- Keep best score and full state ---
          if capped_score > best_score:
              best_score = capped_score
              best_state = {
                  "nested_reps": [np.copy(rep) for rep in self.nested_reps],
                  "capped_node_metrics": metrics_list,
                  "node_score_contributions": np.copy(node_contributions),
                  "l2_before": self.l2_before.copy(),
                  "l2_after": self.l2_after.copy(),
                  "chosen_Gmat": np.copy(self.chosen_Gmat),
                  "inter_layer": self.inter_layer  # already holds weights/bias
              }

      # --- Restore best state ---
      self.nested_reps = best_state["nested_reps"]
      self.capped_node_metrics = best_state["capped_node_metrics"]
      self.node_score_contributions = best_state["node_score_contributions"]
      self.l2_before = best_state["l2_before"]
      self.l2_after = best_state["l2_after"]
      self.chosen_Gmat = best_state["chosen_Gmat"]
      self.inter_layer = best_state["inter_layer"]

      print(f"\nBest Outer Score Achieved: {best_score:.4f}")
      return self.capped_node_metrics



    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base
            y_true = self.synthetic_targets[i][self.best_dim_per_node[i]]
            if len(y_true)<len(y_sel):
                y_true = np.pad(y_true,(0,len(y_sel)-len(y_true)),"constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()

    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep)),rep,color=plt.cm.plasma(rep))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()

    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX, self.D_graph)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i == j:
                        tensor[i, j, :] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                        tensor[i, j, :] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX,self.D_graph)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor
    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX, self.D_graph)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = ['wait','throughput','util','patience']
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            lower = fmt_tensor_bounds[:,:,k,0]
            upper = fmt_tensor_bounds[:,:,k,1]
            mean_vals = (lower+upper)/2
            range_vals = upper-lower
            max_range = range_vals.max() if range_vals.max()>0 else 1.0
            alphas = 0.2 + 0.8 * range_vals/max_range

            im = axes[k].imshow(mean_vals, cmap='viridis', vmin=0, vmax=mean_vals.max())
            for i in range(D):
                for j in range(D):
                    alpha_val = np.clip(1-alphas[i,j],0,1)
                    rect = plt.Rectangle((j-0.5,i-0.5),1,1,color='white',alpha=alpha_val)
                    axes[k].add_patch(rect)
                    axes[k].text(j,i,f"{lower[i,j]:.1f}\n{upper[i,j]:.1f}",ha='center',va='center',fontsize=8)
            axes[k].set_title(f'FMT Bounds - {key}')
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Mean Metric Value')
        plt.tight_layout()
        plt.show()
    def plot_node_score_contribution(self, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot per-node total score contribution in the SAME STYLE as the FMT plots:
            - uses imshow
            - one panel for: raw, FMT, and total stacked
            - diagonal masked
            - annotated cells
            - node contribution highlighted like your FMT code
        """
        # ---------------------------------------------------------------------
        # 1. Collect node contributions from run_outer()
        # ---------------------------------------------------------------------
        _, _, node_contributions = self.run_outer()
        node_contributions = np.array(node_contributions)
        D = len(node_contributions)

        # ---------------------------------------------------------------------
        # 2. Recompute FMT influence (same style as your FMT plots)
        # ---------------------------------------------------------------------
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        total_tensor = fuzzy_tensor.sum(axis=2)           # sum over metrics
        fmt_tensor = total_tensor.copy()
        np.fill_diagonal(fmt_tensor, 0)                   # mask diagonal

        fmt_per_node = fmt_tensor.sum(axis=1)             # row sum
        raw_per_node = node_contributions - fmt_per_node  # everything else

        # Construct matrices for plotting (D×D)
        raw_matrix = np.zeros((D, D))
        fmt_matrix = fmt_tensor
        total_matrix = raw_matrix + fmt_matrix            # raw only on diagonal? no → distribute raw as row diag
        np.fill_diagonal(raw_matrix, raw_per_node)
        total_matrix = raw_matrix + fmt_matrix

        # ---------------------------------------------------------------------
        # 3. Plot - 3 subplots in SAME STYLE as FMT panels
        # ---------------------------------------------------------------------
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))

        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "FMT Interaction Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):

            im = ax.imshow(mat, cmap='viridis', vmin=np.min(mat), vmax=np.max(mat))

            # annotate values
            for i in range(D):
                for j in range(D):
                    val = mat[i, j]
                    ax.text(j, i, f"{val:.2f}", ha='center',
                            va='center', color='white', fontsize=8)

            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04,
                    label='Contribution Value')

        plt.tight_layout()
        plt.show()



# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = FMT(
        candidate_dims, D_graph,
        inner_archive_size, inner_offspring,
        outer_archive_size, outer_offspring,
        synthetic_targets,
        inner_learning, gamma_interlayer=gamma_interlayer,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_nested_activations()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)

# Plot as heatmaps
    optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=True)
    optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    optimizer.plot_node_score_contribution()
    optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    import networkx as nx
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
D_graph = len(optimizer.nested_reps)
tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

# ---------------- Outer nodes (hubs) ----------------
G_outer = nx.DiGraph()
for i in range(D_graph):
    G_outer.add_node(i)
for i in range(D_graph):
    for j in range(D_graph):
        if i != j and np.any(tensor[i,j,:] != 0):
            # Shift to signed weights: 0.5 -> 0, <0.5 negative, >0.5 positive
            mean_weight = 2 * (np.mean(tensor[i,j,:]) - 0.5)
            G_outer.add_edge(i, j, weight=mean_weight)

# Outer spring layout
pos_outer_2d = nx.circular_layout(G_outer, scale=5)
pos_outer = np.array([[x, y, 0] for x, y in pos_outer_2d.values()])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# Plot outer nodes
for i in range(D_graph):
    ax.scatter(*pos_outer[i], s=300, color='skyblue')
    ax.text(*pos_outer[i], f'Node {i}', color='black')

# Plot outer edges with positive/negative colors

for i, j, data in G_outer.edges(data=True):
    x_vals = [pos_outer[i,0], pos_outer[j,0]]
    y_vals = [pos_outer[i,1], pos_outer[j,1]]
    z_vals = [pos_outer[i,2], pos_outer[j,2]]

    # Positive = bright green, Negative = bright red
    color = 'green' if data['weight'] > 0 else 'red'
    linewidth = 2 + 4*abs(data['weight'])  # scale width by magnitude
    ax.plot(x_vals, y_vals, z_vals, color=color, linewidth=linewidth)
# ---------------- Inner FCMs (small circular around hub) ----------------
for i, rep in enumerate(optimizer.nested_reps):
    dims = len(rep)
    angle = np.linspace(0, 2*np.pi, dims, endpoint=False)
    radius = 0.8  # small circle
    xs = pos_outer[i,0] + radius * np.cos(angle)
    ys = pos_outer[i,1] + radius * np.sin(angle)
    zs = pos_outer[i,2] + rep  # activation as height

    # Plot inner nodes
    ax.scatter(xs, ys, zs, c=rep, cmap='plasma', s=50)

    # Connect inner nodes in circle
    for k in range(dims):
        ax.plot([xs[k], xs[(k+1)%dims]], [ys[k], ys[(k+1)%dims]], [zs[k], zs[(k+1)%dims]], color='gray', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Activation')
ax.set_title('Outer Nodes with Inner FCMs (Signed correlations)')
plt.show()


